In [191]:
import pandas as pd
import numpy as np
import os,re,sqlite3, time, datetime

In [9]:
pd.set_option('max_columns',1000)
pd.set_option('max_row',300)

In [176]:
os.listdir()

['version_df.csv',
 '.git',
 'baseinfo_pub.csv',
 'intro',
 'baseinfo_pub2.csv',
 'baseinfo_df.csv',
 'update',
 'aso20190710.db',
 'mergedata20190708.ipynb',
 'version.csv',
 'asodata',
 'merpkg1zuizhong.csv',
 '.ipynb_checkpoints',
 'publish_df.csv']

# 1处理basinfo_pub

## 后爬取的数据表

In [33]:
df1 = pd.read_csv('baseinfo_pub.csv',header = None)
df1 = df1.drop([18],axis =1)
print(df1.shape)

(1674, 22)


In [34]:
df1.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22
0,500619796,捕鱼之海底捞,游戏,免费,2017-06-12,"Sun Ground Co., Ltd.","Sun Ground Co., Ltd.",游戏、娱乐、动作游戏、街机游戏,2012年03月08日,2017年06月12日,com.sg.iphone.fishx,3.8.1,60.15 MB,是,NaN,需要 iOS 7.0 或更高版本。与 iPhone、iPad 和 iPod touch 兼容。,"简体中文, 英语",12+,"Sun Ground Co., Ltd.",9,王者捕鱼|魔盗守卫战--蠢萌消除激情塔防最佳组合！|恶魔来了 -- 七龙纪系列|Legend...,1070819686|946824755|514869891|1077664351|5461...
1,1114765349,神魔圣域—跨国天梯争夺百万悬赏,游戏,免费,2019-05-30,"Beijing Kunlun Online Tech Co., Ltd.",昆仑在线,游戏、动作游戏、角色扮演游戏,2016年06月27日,2019年05月30日,com.kunlun.smsycn,1.107.01,613.06 MB,是,NaN,需要 iOS 8.0 或更高版本。与 iPhone、iPad 和 iPod touch 兼容。,英文,9+,昆仑在线,4,无双剑姬-无锁定3D真动作手游|师父有妖气|艾尔战记（艾尔之光官方授权手游）|艾尔战记-艾尔...,920689697|1021522658|1059976078|1315519379


## 先爬取的数据表

In [37]:
df2 = pd.read_csv('baseinfo_pub2.csv')
df2 = df2.drop(['Unnamed: 0'],axis = 1)

In [39]:
df2.head(2)

,appleid,appName,appType,price,latestVersion,developerFirm,developer,tags,releaseDate,lastestDate,bundleId,lastestVer,size,payInApp,support,compatibility,lang,contentRank,artistname,samepubappnum,samepubapplist,samepubapplistid
0,1097465856,劲舞团-2周年福利大放送,游戏,免费,2018-10-04,"Hangzhou NetEase Leihuo Technology Co., Ltd.",网易移动游戏,游戏、音乐、模拟游戏,2016年09月14日,2018年10月04日,com.netease.jwt,2.4.2,2048 MB,是,NaN,需要 iOS 7.0 或更高版本。与 iPhone、iPad 和 iPod touch 兼容。,简体中文,12+,网易移动游戏,121,F1 Race Stars|Raven:掠夺者(网易)-感受钢铁战斗的激情|一起消消毒|大唐...,941547027|1007987503|838812226|980496876|93229...
1,1204158467,逃脱13间X密室「已下架」,游戏,免费,2017-02-13,NaN,Shuzhen Song,游戏、角色扮演游戏、文字游戏,2017年02月13日,2017年02月13日,studio.dream.esc13xro,1.0.111,61.96 MB,是,NaN,需要 iOS 6.0 或更高版本。与 iPhone、iPad 和 iPod touch 兼容。,"日语, 简体中文, 繁体中文, 英语",9+,Shuzhen Song,261,ARPG 超神传奇|ARPG 超神传奇豪华版|逃脱游戏:有趣的12间密室|逃脱游戏:有趣的1...,1142412438|1142412442|1141203942|1141203951|11...


In [40]:
print(df2.shape)

(12685, 22)


In [41]:
columns = df2.columns.tolist()
print(len(columns))
print(columns)

22
['appleid', 'appName', 'appType', 'price', 'latestVersion', 'developerFirm', 'developer', 'tags', 'releaseDate', 'lastestDate', 'bundleId', 'lastestVer', 'size', 'payInApp', 'support', 'compatibility', 'lang', 'contentRank', 'artistname', 'samepubappnum', 'samepubapplist', 'samepubapplistid']


## 为后爬取的数据表添加列名

In [42]:
df1.columns = columns

In [43]:
df1.head()

,appleid,appName,appType,price,latestVersion,developerFirm,developer,tags,releaseDate,lastestDate,bundleId,lastestVer,size,payInApp,support,compatibility,lang,contentRank,artistname,samepubappnum,samepubapplist,samepubapplistid
0,500619796,捕鱼之海底捞,游戏,免费,2017-06-12,"Sun Ground Co., Ltd.","Sun Ground Co., Ltd.",游戏、娱乐、动作游戏、街机游戏,2012年03月08日,2017年06月12日,com.sg.iphone.fishx,3.8.1,60.15 MB,是,NaN,需要 iOS 7.0 或更高版本。与 iPhone、iPad 和 iPod touch 兼容。,"简体中文, 英语",12+,"Sun Ground Co., Ltd.",9,王者捕鱼|魔盗守卫战--蠢萌消除激情塔防最佳组合！|恶魔来了 -- 七龙纪系列|Legend...,1070819686|946824755|514869891|1077664351|5461...
1,1114765349,神魔圣域—跨国天梯争夺百万悬赏,游戏,免费,2019-05-30,"Beijing Kunlun Online Tech Co., Ltd.",昆仑在线,游戏、动作游戏、角色扮演游戏,2016年06月27日,2019年05月30日,com.kunlun.smsycn,1.107.01,613.06 MB,是,NaN,需要 iOS 8.0 或更高版本。与 iPhone、iPad 和 iPod touch 兼容。,英文,9+,昆仑在线,4,无双剑姬-无锁定3D真动作手游|师父有妖气|艾尔战记（艾尔之光官方授权手游）|艾尔战记-艾尔...,920689697|1021522658|1059976078|1315519379
2,486314228,星际战争:异形入侵,游戏,免费,2018-11-21,iFreyr Games,Freyr Games,游戏、动作游戏、街机游戏,2012年01月22日,2018年11月21日,com.ifreyr.starwarfare,2.96,400.67 MB,是,NaN,需要 iOS 8.0 或更高版本。与 iPhone、iPad 和 iPod touch 兼容。,英语,12+,Freyr Games,6,狂奔总动员|斯巴达克斯：战神|僵尸之王|星际战争 2：初次反击|边境|神魔召唤师,547135385|965099010|698009737|888211509|981005...
3,1114277380,闲来跑得快-正宗湖南扑克牌玩法合集「已下架」,游戏,免费,2017-12-12,Chenrong Lu,Chenrong Lu,游戏、娱乐场游戏、扑克牌游戏,2016年05月28日,2017年12月12日,com.xianlai.mahjonghnpdk,1.0.31,97.46 MB,是,NaN,需要 iOS 7.0 或更高版本。与 iPhone、iPad 和 iPod touch 兼容。,英语,17+,Chenrong Lu,0,,
4,1108802570,热血江湖- 双喜临门 百变飞升,游戏,免费,2019-06-14,Beijing Zhong Qing Long Tu Network Technology ...,龙图游戏,游戏、角色扮演游戏、动作游戏,2017年04月09日,2019年06月14日,com.longtugame.rxjh,1.0.98,1148.29 MB,是,NaN,需要 iOS 8.0 或更高版本。与 iPhone、iPad 和 iPod touch 兼容。,简体中文,12+,龙图游戏,7,山口山战记-奥丁王座|媚三国-即时战斗卡牌|小冰冰传奇——命运塔罗牌！|我的文明|小熊爱消除...,1105367722|821382144|793077082|1115108313|1073...


## 合并两部分数据

In [44]:
dfm = df1.append(df2)

In [49]:
dfm.shape

(14359, 22)

In [47]:
dfm1 = dfm.drop_duplicates(subset = ['appleid'])

In [48]:
dfm1.shape

(13088, 22)

# 2构造增加文本的函数

In [70]:
def read_content(appleid = None,file = None):
    if appleid != None:
        filename = 'intro{}.txt'.format(appleid)
        path = 'intro'
    else:
        filename = file
        path = 'update'
    if os.path.exists(os.path.join(path,path,filename)):
        try:
            with open(os.path.join(path,path,filename),'r') as f:
                content = f.read()
            content = re.sub(r'\s','',content)
        except:
            content = np.nan
    elif os.path.exists(os.path.join(path,'{}1'.format(path),filename)):
        try:
            with open(os.path.join(path,'{}1'.format(path),filename),'r') as f:
                content = f.read()
            content = re.sub(r'\s','',content)
        except:
            content = np.nan
    elif os.path.exists(os.path.join(path,'{}2'.format(path),filename)):
        try:
            with open(os.path.join(path,'{}2'.format(path),filename),'r') as f:
                content = f.read()
            content = re.sub(r'\s','',content)
        except:
            content = np.nan
    else:
        content = np.nan
    return content

In [101]:
print(read_content(file = '500619796v3_8_1.txt'))

1.bug修正2.游戏体验优化


# 3version表

## 后爬取的version

In [74]:
dfv1 = pd.read_csv('version.csv',header = None)

In [82]:
print(dfv1.shape)

(28365, 6)


In [75]:
dfv1.head(2)

,0,1,2,3,4,5
0,500619796,2017年06月12日,1.497197e+09,3.8.1,v3.8.1,500619796v3_8_1.txt
1,500619796,2017年03月25日,1.490371e+09,3.8.0,v3.8.0,500619796v3_8_0.txt


## 先爬的version

In [80]:
dfv2 = pd.read_csv('version_df.csv')
dfv2 = dfv2.drop(['Unnamed: 0'], axis = 1)

In [83]:
print(dfv2.shape)

(100455, 6)


In [81]:
dfv2.head(2)

,appleid,verDate,timestamp,versionTitle,versionTitle2,filename
0,1097465856,2018年10月04日,1.538582e+09,2.4.2,v2.4.2,1097465856v2_4_2.txt
1,1097465856,2018年09月27日,1.537978e+09,2.4.0,v2.4.0,1097465856v2_4_0.txt


## 给后爬的添加columns

In [86]:
columnsv = dfv2.columns.tolist()
print(len(columnsv))
print(columnsv)

6
['appleid', 'verDate', 'timestamp', 'versionTitle', 'versionTitle2', 'filename']


In [87]:
dfv1.columns = columnsv

In [88]:
dfv1.shape

(28365, 6)

In [89]:
dfv1.head(2)

,appleid,verDate,timestamp,versionTitle,versionTitle2,filename
0,500619796,2017年06月12日,1.497197e+09,3.8.1,v3.8.1,500619796v3_8_1.txt
1,500619796,2017年03月25日,1.490371e+09,3.8.0,v3.8.0,500619796v3_8_0.txt


## 合并两部分数据

In [91]:
dfvm = dfv1.append(dfv2)

In [92]:
dfvm.shape

(128820, 6)

In [93]:
dfvm1 = dfvm.drop_duplicates(subset = ['filename'])

In [94]:
dfvm1.shape

(106114, 6)

# 小结
目前获得两部分数据

**baseinfro_pub** 数据存储在dataframe：dfm1

**version** 数据存储在dataframe：dfvm1

函数：read_content，两个参数applid，file，只需要输入一个参数就可以


# 形成文本df

1形成一个两列（appleid，introcontent）的dataframe: **df_id_con** 并且整合到dfm1

2形成一个两列（filename, versioncontent）的dataframe: **df_ver_con**并且整合到dfvm1

## 1 buffer行函数

In [104]:
# 新建缓存行
def gen_buffer_line(appleid = None, file = None):
    if appleid != None:
        temp_dict = {'appleid':appleid,
                    'introcontent':read_content(appleid = appleid)}
    else:
        temp_dict = {'filename':file,
                    'vercontent':read_content(file = file)}
    temp_df = pd.DataFrame(data = temp_dict, index = [0])
    return temp_df                    

In [105]:
# test1
gen_buffer_line(appleid = 500619796)

,appleid,introcontent
0,500619796,与美人鱼畅游华丽逼真的海底世界!可爱萌宠个个身怀绝技!让你欲罢不能的捕鱼游戏!唯一不带广告的...


In [115]:
# test2
gen_buffer_line(file = '500619796v3_8_1.txt')

,filename,vercontent
0,500619796v3_8_1.txt,1.bug修正2.游戏体验优化


## 2 构造**df_id_con**

In [116]:
df_id_con = pd.DataFrame(columns = ['appleid','introcontent'])
for r in dfm1.itertuples():
    appleid = getattr(r,'appleid')
    df_id_con = df_id_con.append(gen_buffer_line(appleid = appleid),ignore_index = True)

In [117]:
df_id_con.shape

(13088, 2)

In [118]:
df_id_con.head()

,appleid,introcontent
0,500619796,与美人鱼畅游华丽逼真的海底世界!可爱萌宠个个身怀绝技!让你欲罢不能的捕鱼游戏!唯一不带广告的...
1,1114765349,最强5人天梯战队赛上线，只要达到120级并凑齐5名小伙伴（最多5名）就可以参加“最强战队”比...
2,486314228,v2.93更新内容-新增两把酷炫武器-新增一套霸气盔甲-新增一个实用背包-修复bug促销期间...
3,1114277380,闲来跑得快，正宗本地玩法，随时随地，轻松开局！【游戏介绍】1、最正宗：深入当地调研，实时调整...
4,1108802570,热血江湖手游是由官方唯一正版授权，龙图游戏发行青春武侠MMORPG手游。清新明亮的游戏场景，...


## 3 构造df_ver_con

In [119]:
df_ver_con = pd.DataFrame(columns = ['filename','vercontent'])
for r in dfvm1.itertuples():
    file = getattr(r,'filename')
    df_ver_con = df_ver_con.append(gen_buffer_line(file = file),ignore_index = True)

In [120]:
df_ver_con.shape

(106114, 2)

In [121]:
df_ver_con.head()

,filename,vercontent
0,500619796v3_8_1.txt,1.bug修正2.游戏体验优化
1,500619796v3_8_0.txt,1.bug修正2.游戏体验优化
2,500619796v3_7_7.txt,庆新版本更新，三重好礼送不停！第一重：任务奖励翻倍送！所有任务的奖励直接翻倍！任务重置时间缩...
3,500619796vV3_7_6.txt,修复游戏部分闪退bug
4,500619796v3_7_5.txt,修复成就系统bug


## 4 将df_id_con和df_ver_con整合到dfm1和dfvm1

In [123]:
dfm2 = dfm1.merge(df_id_con,how = 'left', on = 'appleid')

In [126]:
print(dfm1.shape)
print(df_id_con.shape)
print(dfm2.shape)

(13088, 22)
(13088, 2)
(13088, 23)


In [127]:
dfvm2 = dfvm1.merge(df_ver_con, how = 'left', on = 'filename')

In [129]:
print(dfvm1.shape)
print(df_ver_con.shape)
print(dfvm2.shape)

(106114, 6)
(106114, 2)
(106114, 7)


## 小结
所有数据都存储在两个dataframe中一个为dfm2 和dfvm2

# 补充安卓包名到dfm2表，dfm2表中日期转化为时间戳

In [201]:
mergedata = pd.read_csv('merpkg1zuizhong.csv',encoding = 'gbk')

In [202]:
mergedata.head()

,Unnamed: 0,type_code,pkgName,type,appleid,qimaiId,bundleId,appName
0,1777,T201300,a5game.cn.Feedingfish,休闲游戏,1038551692,NaN,NaN,NaN
1,7050,T200500,a5game.leidian2_dx_5a,飞行游戏,997429574,NaN,NaN,NaN
2,7642,T200400,Adrenaline.Crew,赛车游戏,NaN,NaN,NaN,NaN
3,41,T201900,age.of.civilizations.asia.jakowski,策略游戏,NaN,NaN,NaN,NaN
4,5675,T200100,air.AndBadao,动作游戏,NaN,NaN,火柴人部落,NaN


In [203]:
md1 = mergedata.dropna(subset = ['appleid'])
md1 = md1.drop(['appName','bundleId','Unnamed: 0'], axis = 1)

In [204]:
print(mergedata.shape)
print(md1.shape)

(8031, 8)
(4910, 5)


In [205]:
md1.head()

,type_code,pkgName,type,appleid,qimaiId
0,T201300,a5game.cn.Feedingfish,休闲游戏,1038551692,NaN
1,T200500,a5game.leidian2_dx_5a,飞行游戏,997429574,NaN
5,T201300,air.au.com.metro.DumbWaysToDie,休闲游戏,639930688,NaN
6,T201300,air.au.com.metro.DumbWaysToDie2,休闲游戏,929346489,NaN
8,T201700,air.com.a77375.klhxtaomii,模拟经营,1230337086,NaN


In [206]:
dfm3 = dfm2.merge(md1,on = 'appleid',how = 'left')

In [207]:
print(dfm2.shape)
print(dfm3.shape)

(13088, 23)
(13088, 27)


In [208]:
dfm3.columns

Index(['appleid', 'appName', 'appType', 'price', 'latestVersion',
       'developerFirm', 'developer', 'tags', 'releaseDate', 'lastestDate',
       'bundleId', 'lastestVer', 'size', 'payInApp', 'support',
       'compatibility', 'lang', 'contentRank', 'artistname', 'samepubappnum',
       'samepubapplist', 'samepubapplistid', 'introcontent', 'type_code',
       'pkgName', 'type', 'qimaiId'],
      dtype='object')

## 测试用

In [189]:
test = dfm3[['appleid','releaseDate']]

In [190]:
test.head()

,appleid,releaseDate
0,500619796,2012年03月08日
1,1114765349,2016年06月27日
2,486314228,2012年01月22日
3,1114277380,2016年05月28日
4,1108802570,2017年04月09日


In [192]:
def changedate(datestring):
    date_fomat = time.strptime(datestring, '%Y年%m月%d日')
    timestamp = int(time.mktime(date_fomat))
    return timestamp
    

In [193]:
# test
changedate('2012年03月08日')

1331136000

In [197]:
test['timestamp'] = test['releaseDate'].apply(changedate)

/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [195]:
test.head()

,appleid,releaseDate,timestamp
0,500619796,2012年03月08日,1331136000
1,1114765349,2016年06月27日,1466956800
2,486314228,2012年01月22日,1327161600
3,1114277380,2016年05月28日,1464364800
4,1108802570,2017年04月09日,1491667200


In [210]:
dfm4 = dfm3.copy()
dfm4['rdtimestamp'] = dfm4['releaseDate'].apply(changedate)

In [211]:
print(dfm2.shape)
print(dfm3.shape)
print(dfm4.shape)

(13088, 23)
(13088, 27)
(13088, 28)


In [213]:
dfm4.head(2)

,appleid,appName,appType,price,latestVersion,developerFirm,developer,tags,releaseDate,lastestDate,bundleId,lastestVer,size,payInApp,support,compatibility,lang,contentRank,artistname,samepubappnum,samepubapplist,samepubapplistid,introcontent,type_code,pkgName,type,qimaiId,rdtimestamp
0,500619796,捕鱼之海底捞,游戏,免费,2017-06-12,"Sun Ground Co., Ltd.","Sun Ground Co., Ltd.",游戏、娱乐、动作游戏、街机游戏,2012年03月08日,2017年06月12日,com.sg.iphone.fishx,3.8.1,60.15 MB,是,NaN,需要 iOS 7.0 或更高版本。与 iPhone、iPad 和 iPod touch 兼容。,"简体中文, 英语",12+,"Sun Ground Co., Ltd.",9,王者捕鱼|魔盗守卫战--蠢萌消除激情塔防最佳组合！|恶魔来了 -- 七龙纪系列|Legend...,1070819686|946824755|514869891|1077664351|5461...,与美人鱼畅游华丽逼真的海底世界!可爱萌宠个个身怀绝技!让你欲罢不能的捕鱼游戏!唯一不带广告的...,NaN,NaN,NaN,NaN,1331136000
1,1114765349,神魔圣域—跨国天梯争夺百万悬赏,游戏,免费,2019-05-30,"Beijing Kunlun Online Tech Co., Ltd.",昆仑在线,游戏、动作游戏、角色扮演游戏,2016年06月27日,2019年05月30日,com.kunlun.smsycn,1.107.01,613.06 MB,是,NaN,需要 iOS 8.0 或更高版本。与 iPhone、iPad 和 iPod touch 兼容。,英文,9+,昆仑在线,4,无双剑姬-无锁定3D真动作手游|师父有妖气|艾尔战记（艾尔之光官方授权手游）|艾尔战记-艾尔...,920689697|1021522658|1059976078|1315519379,最强5人天梯战队赛上线，只要达到120级并凑齐5名小伙伴（最多5名）就可以参加“最强战队”比...,NaN,NaN,NaN,NaN,1466956800


# 将数据转化到sqlite中

In [167]:
conn = sqlite3.connect('aso20190710.db')
c = conn.cursor()

## 1建立baseinfo表

In [135]:
dfm2.head(1)

,appleid,appName,appType,price,latestVersion,developerFirm,developer,tags,releaseDate,lastestDate,bundleId,lastestVer,size,payInApp,support,compatibility,lang,contentRank,artistname,samepubappnum,samepubapplist,samepubapplistid,introcontent
0,500619796,捕鱼之海底捞,游戏,免费,2017-06-12,"Sun Ground Co., Ltd.","Sun Ground Co., Ltd.",游戏、娱乐、动作游戏、街机游戏,2012年03月08日,2017年06月12日,com.sg.iphone.fishx,3.8.1,60.15 MB,是,NaN,需要 iOS 7.0 或更高版本。与 iPhone、iPad 和 iPod touch 兼容。,"简体中文, 英语",12+,"Sun Ground Co., Ltd.",9,王者捕鱼|魔盗守卫战--蠢萌消除激情塔防最佳组合！|恶魔来了 -- 七龙纪系列|Legend...,1070819686|946824755|514869891|1077664351|5461...,与美人鱼畅游华丽逼真的海底世界!可爱萌宠个个身怀绝技!让你欲罢不能的捕鱼游戏!唯一不带广告的...


In [138]:
','.join(dfm2.columns.tolist())

'appleid,appName,appType,price,latestVersion,developerFirm,developer,tags,releaseDate,lastestDate,bundleId,lastestVer,size,payInApp,support,compatibility,lang,contentRank,artistname,samepubappnum,samepubapplist,samepubapplistid,introcontent'

In [146]:
len(dfm2.columns.tolist())

23

In [168]:
try:
    c.execute('''create table baseinfo
    (appleid integer primary key,
    appName varchar(50),
    appType varchar(10),
    price varchar(10),
    latestVersion varchar(20),
    developerFirm varchar(50),
    developer varchar(50),
    tags varchar(50),
    releaseDate varchar(20),
    lastestDate varchar(20),
    bundleId varchar(50),
    lastestVer varchar(20),
    size varchar(10),
    payInApp varchar(10),
    support varchar(20),
    compatibility varchar(50),
    lang varchar(20),
    contentRank varchar(10),
    artistname varchar(50),
    samepubappnum integer,
    samepubapplist text,
    samepubapplistid text,
    introcontent text)''')
    conn.commit()
    print('baseinfo create')
except:
    print('baseinfo exits')

baseinfo create


## 2建立version表

In [141]:
dfvm2.head(1)

,appleid,verDate,timestamp,versionTitle,versionTitle2,filename,vercontent
0,500619796,2017年06月12日,1.497197e+09,3.8.1,v3.8.1,500619796v3_8_1.txt,1.bug修正2.游戏体验优化


In [142]:
','.join(dfvm2.columns.tolist())

'appleid,verDate,timestamp,versionTitle,versionTitle2,filename,vercontent'

In [169]:
try:
    c.execute('''create table version
    (appleid integer,
    verDate varchar(20),
    timestamp integer,
    versionTitle varchar(20),
    versionTitle2 varchar(20),
    filename varchar(30) primary key,
    vercontent text)''')
    conn.commit()
    print('version create')
except:
    print('version exists')

version create


## 3建立baseinfo_td表

In [221]:
try:
    c.execute('''create table baseinfo_td
    (appleid integer primary key,
    appName varchar(50),
    appType varchar(10),
    price varchar(10),
    latestVersion varchar(20),
    developerFirm varchar(50),
    developer varchar(50),
    tags varchar(50),
    releaseDate varchar(20),
    lastestDate varchar(20),
    bundleId varchar(50),
    lastestVer varchar(20),
    size varchar(10),
    payInApp varchar(10),
    support varchar(20),
    compatibility varchar(50),
    lang varchar(20),
    contentRank varchar(10),
    artistname varchar(50),
    samepubappnum integer,
    samepubapplist text,
    samepubapplistid text,
    introcontent text,
    type_code_TD varchar(20),
    pkgName varchar(50),
    type_TD varchar(20),
    qimaiId integer,
    rdtimestamp integer)''')
    conn.commit()
    print('baseinfo_td create')
except:
    print('baseinfo_td exits')

baseinfo_td create


# 将数据存入sqlite

In [173]:
for r in dfvm2.itertuples(index = False):
    temp = tuple([i for i in r])
    try:
        c.execute('insert into version values (?,?,?,?,?,?,?)',temp)
        conn.commit()
    except:
        print('error: {}'.format(getattr(r,'filename')))

error: 500619796v3_8_1.txt


In [174]:
for r in dfm2.itertuples(index = False):
    temp = tuple([i for i in r])
    try:
        c.execute('insert into baseinfo values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',temp)
        conn.commit()
    except:
        print('error: {}'.format(getattr(r,'appleid')))

error: 500619796


In [224]:
for r in dfm4.itertuples(index = False):
    temp = tuple([i for i in r])
    try:
        c.execute('insert into baseinfo_td values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',temp)
        conn.commit()
    except:
        print('error: {}'.format(getattr(r,'appleid')))

In [225]:
jijijiji = '?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?'
lilililiil = jijijiji.split(',')
len(lilililiil)

28

In [227]:
a = c.execute('select count(*) from baseinfo_td')

In [228]:
print(a)